<a href="https://colab.research.google.com/github/madarasw/Machine-Learning/blob/main/Finetune_TinyLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers peft accelerate datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.6 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True  # reduces memory use (QLoRA style)
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
from datasets import Dataset

data = {
    "instruction": [
        "What is inflation?",
        "Explain GDP in simple terms.",
        "What causes unemployment?"
    ],
    "response": [
        "Inflation is the general increase in prices over time.",
        "GDP is the total value of goods and services produced in a country.",
        "Unemployment happens when people who want to work cannot find jobs."
    ]
}

dataset = Dataset.from_dict(data)

In [5]:
def format(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n### Response:\n{example['response']}"
    }

dataset = dataset.map(format)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [6]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, config)

In [10]:
def format(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n### Response:\n{example['response']}"
    }

dataset = dataset.map(format)

def tokenize(example):
    outputs = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    outputs["labels"] = outputs["input_ids"].copy()  # ✅ add labels for causal LM
    return outputs

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [18]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="tinyllama-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    remove_unused_columns=False,  # keep all custom fields
    report_to="none" # Disable wandb logging
)

In [19]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=2, training_loss=17.42969512939453, metrics={'train_runtime': 3.4481, 'train_samples_per_second': 1.74, 'train_steps_per_second': 0.58, 'total_flos': 19088894066688.0, 'train_loss': 17.42969512939453, 'epoch': 2.0})

In [20]:
model.save_pretrained("tinyllama-lora")
tokenizer.save_pretrained("tinyllama-lora")

('tinyllama-lora/tokenizer_config.json',
 'tinyllama-lora/special_tokens_map.json',
 'tinyllama-lora/chat_template.jinja',
 'tinyllama-lora/tokenizer.model',
 'tinyllama-lora/added_tokens.json',
 'tinyllama-lora/tokenizer.json')

In [21]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "### Instruction:\nExplain GDP.\n### Response:\n"
print(pipe(prompt, max_new_tokens=50)[0]["generated_text"])

Device set to use cuda:0


### Instruction:
Explain GDP.
### Response:
GDP (Gross Domestic Product) is the total value of all goods and services produced in a particular economy in a given year. It is a measure of the economic strength and productive capacity of a country. It reflects the economic performance


In [22]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "### Instruction:\nExplain CPI.\n### Response:\n"
print(pipe(prompt, max_new_tokens=50)[0]["generated_text"])

Device set to use cuda:0


### Instruction:
Explain CPI.
### Response:
CPI or Constant Pricing Index measures the average decrease in prices during a specific time period. It is calculated by dividing the number of sales during a specific time period by the average number of sales during the same period. This method helps to identify


In [23]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "### Instruction:\nExplain Madara.\n### Response:\n"
print(pipe(prompt, max_new_tokens=50)[0]["generated_text"])

Device set to use cuda:0


### Instruction:
Explain Madara.
### Response:
Madara was a powerful and mysterious sorcerer who had been banished from the kingdom of Altsin. He had been accused of several crimes, including the assassination of the king, plotting against the kingdom's rul


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
prompt = "### Instruction:\nExplain GDP.\n### Response:\n"
print(pipe(prompt, max_new_tokens=50)[0]["generated_text"])